In [1]:
path = '/Users/mgurmendez/Downloads/Social_Network_Ads 2.csv'


In [2]:
from pyspark.context import SparkContext

sc = SparkContext('local', 'test')

In [4]:
from pyspark.sql import SQLContext

spark = SQLContext(sc)

In [7]:
df = spark.read.csv(path, header=True)

In [9]:
df.show(5)

+--------+------+---+---------------+---------+
| User ID|Gender|Age|EstimatedSalary|Purchased|
+--------+------+---+---------------+---------+
|15624510|  Male| 19|          19000|        0|
|15810944|  Male| 35|          20000|        0|
|15668575|Female| 26|          43000|        0|
|15603246|Female| 27|          57000|        0|
|15804002|  Male| 19|          76000|        0|
+--------+------+---+---------------+---------+
only showing top 5 rows



In [10]:
df.createOrReplaceTempView("purchases")

In [11]:
spark.sql("select count(distinct EstimatedSalary) from purchases").show()

+-------------------------------+
|count(DISTINCT EstimatedSalary)|
+-------------------------------+
|                            117|
+-------------------------------+



In [12]:
spark.sql("select count(distinct Age) from purchases").show()

+-------------------+
|count(DISTINCT Age)|
+-------------------+
|                 43|
+-------------------+



In [ ]:
# P(P=1 | G=Male, A=70, S=18000) = ?
# P(G=Male, A=70, S=18000 | P=0) y P(G=Male, A=70, S=18000 | P=1)

In [13]:
spark.sql("select * from purchases where Age=70 and EstimatedSalary=18000 and Gender='Male'").show()

+-------+------+---+---------------+---------+
|User ID|Gender|Age|EstimatedSalary|Purchased|
+-------+------+---+---------------+---------+
+-------+------+---+---------------+---------+



In [14]:
spark.sql("select * from purchases where Age=19 and EstimatedSalary=19000 and Gender='Male'").show()

+--------+------+---+---------------+---------+
| User ID|Gender|Age|EstimatedSalary|Purchased|
+--------+------+---+---------------+---------+
|15624510|  Male| 19|          19000|        0|
+--------+------+---+---------------+---------+



In [21]:
spark.sql("select count(*) from purchases where Purchased=0").show()

+--------+
|count(1)|
+--------+
|     257|
+--------+



In [ ]:
# P(G=Male, A=19, S=19000 | P=0) = 1/257
# P(G=Male, A=19, S=19000 | P=1) = 0/(400-257)
# P(G=Male, A=19, S=19000) = 1/400

In [31]:
dfs = spark.sql("select Gender,if(Age<28,'low','high') as AgeBucket," +
                "if(EstimatedSalary<30000,'low','high') as SalaryBucket,Purchased from purchases")

In [36]:
dfs.createOrReplaceTempView("dpurchases")

In [32]:
dfs.show()

+------+---------+------------+---------+
|Gender|AgeBucket|SalaryBucket|Purchased|
+------+---------+------------+---------+
|  Male|      low|         low|        0|
|  Male|     high|         low|        0|
|Female|      low|        high|        0|
|Female|      low|        high|        0|
|  Male|      low|        high|        0|
|  Male|      low|        high|        0|
|Female|      low|        high|        0|
|Female|     high|        high|        1|
|  Male|      low|        high|        0|
|Female|     high|        high|        0|
|Female|      low|        high|        0|
|Female|      low|        high|        0|
|  Male|      low|        high|        0|
|  Male|     high|         low|        0|
|  Male|      low|        high|        0|
|  Male|     high|        high|        0|
|  Male|     high|         low|        1|
|  Male|     high|         low|        1|
|  Male|     high|         low|        1|
|Female|     high|         low|        1|
+------+---------+------------+---

In [ ]:
# P(P=1 | G=Male,A=low,S=low) = P(G=Male,A=high,S=low | P=1) * P(P=1) / P(G=Male,A=high,S=low)



In [43]:
# P(P=1) = 143/400
(spark.sql("select count(*) from purchases where purchased=1").collect(),spark.sql("select count(*) from dpurchases").collect() )

([Row(count(1)=143)], [Row(count(1)=400)])

In [44]:
# P(G=Male,A=high,S=low) = 18/400
spark.sql("select count(*) from dpurchases where gender='Male' and salarybucket='low' and agebucket='high'").show()

+--------+
|count(1)|
+--------+
|      18|
+--------+



In [45]:
#P(G=Male,A=high,S=low | P=1) = 0
spark.sql("select purchased,count(*) from dpurchases where gender='Male' and salarybucket='low' and agebucket='high' group by  purchased").show()

+---------+--------+
|purchased|count(1)|
+---------+--------+
|        0|       8|
|        1|      10|
+---------+--------+



In [ ]:
# P(P=1 | G=Male,A=high,S=low) = P(G=Male,A=high,S=low | P=1) * P(P=1) / P(G=Male,A=high,S=low)
                               = 10/143 * 143/400 / (18/400)

In [47]:
(10/143) * (143/400) / (18/400)

0.5555555555555556

In [ ]:
# P(P=0 | G=Male,A=high,S=low)  = 0.45  

In [ ]:
# P(P=1 | G=Male,A=low,S=low) = 0 :(  :(  :(  :(

In [ ]:
# P(P=1 | G=Male,A=low,S=low)  = P(G=Male,A=low,S=low | P=1) * P(P=1) / P(G=Male,A=low,S=low)

# P(G=Male,A=low,S=low | P=1) = P(G=Male | P=1) * P(A=low | P=1) * P(S=low | P=1)
                             

In [ ]:
# Independencia: P(G=Male,A=low) =  P(G=Male) * P(A=low) 
# Independencia condicional: P(G=Male,A=low | P=1)  P(G=Male | P=1) * P(A=low | P=1) 




In [52]:
spark.sql("select gender,purchased,count(*) from dpurchases group by gender,purchased order by purchased").show()

+------+---------+--------+
|gender|purchased|count(1)|
+------+---------+--------+
|  Male|        0|     130|
|Female|        0|     127|
|  Male|        1|      66|
|Female|        1|      77|
+------+---------+--------+



In [ ]:
# P(G=Male | P=1) = 66 / (66+77) 
# P(G=Male | P=0) = 130 / (130+127) 

In [53]:
spark.sql("select agebucket,purchased,count(*) from dpurchases group by agebucket,purchased order by purchased").show()

+---------+---------+--------+
|agebucket|purchased|count(1)|
+---------+---------+--------+
|     high|        0|     181|
|      low|        0|      76|
|      low|        1|       2|
|     high|        1|     141|
+---------+---------+--------+



In [ ]:
# P(A=low | P=1) = 2/(2+141)
# P(A=low | P=0) = 76/(76+181)

In [54]:
spark.sql("select salarybucket,purchased,count(*) from dpurchases group by salarybucket,purchased order by purchased").show()

+------------+---------+--------+
|salarybucket|purchased|count(1)|
+------------+---------+--------+
|        high|        0|     222|
|         low|        0|      35|
|         low|        1|      18|
|        high|        1|     125|
+------------+---------+--------+



In [ ]:
# P(S=low | P=1) = 18/(18+125)
# P(S=low | P=0) = 35/(35+222)

In [55]:
# P(G=Male,A=low,S=low | P=1) = 0.0008125279776705274
66 / (66+77) * 2/(2+141) * 18/(18+125)

0.0008125279776705274

In [56]:
# P(G=Male,A=low,S=low | P=0) = 0.0008125279776705274
130 / (130+127) * 76/(76+181) * 35/(35+222)

0.020371622459519352

In [57]:
# P(P=1 | G=Male,A=low,S=low)  = P(G=Male,A=low,S=low | P=1) * P(P=1) / P(G=Male,A=low,S=low)
# = 0.0008125279776705274 * 143/400 /  K

# P(P=0 | G=Male,A=low,S=low)  = P(G=Male,A=low,S=low | P=0) * P(P=0) / P(G=Male,A=low,S=low)
#  = 0.020371622459519352 * (400-143)/400 / K


(0.0008125279776705274 * 143/400,  0.020371622459519352 * (400-143)/400  )

(0.0002904787520172136, 0.013088767430241184)

In [ ]:
# Sabemos que P(P=0 | G=Male,A=low,S=low) > P(P=1 | G=Male,A=low,S=low) 
# ESTE PERFIL NO COMPRA!

In [59]:
# P(G=Male,A=low,S=low) 
#   = P(G=Male,A=low,S=low | P=0) * P(P=0) + P(G=Male,A=low,S=low | P=1) * P(P=1)  =
0.020371622459519352  * (400-143)/400 + 0.0008125279776705274 * 143/400

0.013379246182258398

In [60]:
# P(P=1 | G=Male,A=low,S=low) = 
( 0.0008125279776705274 * 143/400 ) / 0.013379246182258398

0.021711144862735544